In [1]:
# Load required packages
library(dplyr,warn.conflicts=F)
library(stringr,warn.conflicts=F)
library(psych,warn.conflicts=F)
library(lplyr,warn.conflicts=F)
library(tidyr,warn.conflicts=F)

In [7]:
# Final ranking of madness
data.teamtourneyranks <- readRDS('TeamTourneyRanking.rds')
# ranking mapping
data.tourneyranks <- read.csv('TourneyRanks.csv')

In [8]:
# load ranking db
data.ranking <- readRDS('../datafiles_rds/MasseyOrdinals.rds')
# keep 2010+, last day
data.ranking %>% 
    filter(Season>2010) %>%
    group_by(Season) %>%
    filter(RankingDayNum==max(RankingDayNum)) %>%
    select(-RankingDayNum) %>%
    ungroup() -> data.lastranking

In [9]:
# Keep only 68 tourney teams
data.lastranking %>%
    inner_join(data.teamtourneyranks,by=c('Season','TeamID')) %>% 
    group_by(Season,SystemName) %>% 
    summarise(n=n()) %>% 
    ungroup() %>% 
    filter(n==68) -> df.systems
data.lastranking %>%
    semi_join(df.systems,by=c('Season','SystemName')) -> df.ranking

In [11]:
df.ranking %>% head
df.systems %>% head

Season,SystemName,TeamID,OrdinalRank
2011,AUS,1102,108
2011,AUS,1103,127
2011,AUS,1104,55
2011,AUS,1105,323
2011,AUS,1106,312
2011,AUS,1107,254


Season,SystemName,n
2011,AUS,68
2011,BIH,68
2011,BOB,68
2011,BPI,68
2011,CNG,68
2011,COL,68


In [14]:
data.matchups <- readRDS('../datafiles_rds/TourneyMatchups.rds')
data.truth <- readRDS('../datafiles_rds/TourneyTrueResults.rds')

In [16]:
data.matchups %>%
    inner_join(df.systems %>% select(-n),by='Season') -> df.matchups

In [26]:
df.matchups %>%
    inner_join(df.ranking,by=c('Season','TeamID.x'='TeamID','SystemName')) %>%
    rename(Rank.x=OrdinalRank) %>%
    inner_join(df.ranking,by=c('Season','TeamID.y'='TeamID','SystemName')) %>%
    rename(Rank.y=OrdinalRank) %>%
    mutate(Pred=ifelse(Rank.x<Rank.y,1,0)) %>%
    mutate(ID=str_c(Season,'_',TeamID.x,'_',TeamID.y)) %>%
    select(ID,Season,SystemName,Pred) -> df.predict

In [27]:
df.predict %>%
    inner_join(data.truth,by='ID') %>%
    mutate(OK=ifelse(Pred==Target,1,0)) -> df.results

In [34]:
df.results %>%
    group_by(Season,SystemName) %>%
    summarise(Tot=sum(OK)) %>%
    arrange(Season,desc(Tot)) %>%
    slice(1:1) %>%
    ungroup()

Season,SystemName,Tot
2011,POM,46
2012,CPR,57
2013,CPR,52
2014,UPS,48
2015,DC,53
2016,LMC,51
2017,DOK,53
2018,MAS,51


In [39]:
df.results %>%
    group_by(Season,SystemName) %>%
    summarise(Tot=sum(OK)) %>%
    arrange(Season,desc(Tot)) %>%
    slice(1:3) %>%
    mutate(rn=row_number()) %>%
    mutate(W=case_when(rn==1 ~ 2, TRUE ~ 1)) %>%
    ungroup() %>%
    select(-Tot,-rn) -> df.masters

In [40]:
df.results %>%
    inner_join(df.masters,by=c('Season','SystemName')) %>% head

ID,Season,SystemName,Pred,Target,OK,W
2011_1155_1452,2011,DC,0,0,1,1
2011_1155_1452,2011,DCI,1,0,0,1
2011_1155_1452,2011,POM,0,0,1,2
2011_1155_1412,2011,DC,0,1,0,1
2011_1155_1412,2011,DCI,1,1,1,1
2011_1155_1412,2011,POM,1,1,1,2


In [46]:
df.results %>%
    inner_join(df.masters,by=c('Season','SystemName')) %>%
    group_by(ID) %>%
    mutate(Tot=sum(OK*W)) %>%
    mutate_which(Tot==2,Pred=1-Pred) %>%
    ungroup() %>%
    filter(W==2) %>%
    mutate(OK=ifelse(Pred==Target,1,0)) %>%
    select(-W,-Tot) %>%
    group_by(Season,SystemName) %>%
    summarise(Tot=sum(OK)) %>%
    arrange(Season,desc(Tot)) %>%
    slice(1:1) %>%
    ungroup()    

Season,SystemName,Tot
2011,POM,46
2012,CPR,51
2013,CPR,49
2014,UPS,49
2015,DC,53
2016,LMC,52
2017,DOK,53
2018,MAS,52


In [48]:
53/67

[1] 0.7910448